In [ ]:
"""libraries required"""
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install timm
!pip install torchattacks

In [2]:
"""connect to google drive to store and retrieve data"""
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Dataset Loader**

In [ ]:
import h5py
import torch
import numpy as np
from torch.utils.data import Dataset, TensorDataset
import torchvision

# transform = torchvision.transforms.Resize((224, 224), antialias=True)
device = "cuda" if torch.cuda.is_available() else "cpu"

# data_list = []
# label_list = []

# labels = {'airport': 0, 'shopping_mall': 1, 'metro_station': 2, 'street_pedestrian': 3, 'public_square': 4,
#           'street_traffic': 5, 'tram': 6, 'bus': 7, 'metro': 8, 'park': 9}

# # Open the HDF5 file
# h5_file_path = '/content/drive/MyDrive/data/datasets/DCASE19/DCASE19.hdf5'
# output_h5_file_path = '/content/drive/MyDrive/data/AdvData/DCASE19.h5'

# with h5py.File(h5_file_path, 'r') as hf:
#     with h5py.File(output_h5_file_path, 'w') as hf_out:
#         # Iterate over the samples in the HDF5 file
#         i = 0
#         for s, r in hf.items():
#             label_name = s.split('-')[0]
#             label_number = labels.get(label_name)

#             tensor_data = torch.tensor(r[:]).float() / 255.0
#             tensor_data = tensor_data.unsqueeze(0)
#             tensor_data = transform(tensor_data)

#             grp = hf_out.create_group(f'sample_{i}')
#             grp.create_dataset('data', data=tensor_data.cpu().numpy())
#             grp.create_dataset('label', data=label_number)
#             i += 1

# print(f'dataset stored at {output_h5_file_path}')

# Load directly from h5

In [12]:
from torch.utils.data import TensorDataset
import h5py
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

data_list = []
label_list = []

# Open the HDF5 file
h5_file_path = '/content/drive/MyDrive/data/AdvData/DCASE19.h5'
with h5py.File(h5_file_path, 'r') as hf:
    # Iterate over the samples in the HDF5 file
    for sample_name in hf.keys():
        data = torch.tensor(hf[sample_name]['data'][:])
        label = hf[sample_name]['label'][()]

        # Append data and label to lists
        data_list.append(data)
        label_list.append(label)

dcase = TensorDataset(torch.stack(data_list), torch.tensor(label_list))

In [14]:
len(dcase)

13859

**Function to train the model and calculate metrics**

In [ ]:
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import random_split
import torch.nn.functional as F
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score
from sklearn.manifold import TSNE
import seaborn as sns

# function to train the model
def train_model(model, dataset, learning_rate=2e-4, num_epochs=10, batch_size=128, stepSize=10, gamma=0.1, validation_split=0.2, patience=3):
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    scheduler = StepLR(optimizer, step_size=stepSize, gamma=gamma)

    # Split dataset into training and validation sets
    num_samples = len(dataset)
    num_validation_samples = int(validation_split * num_samples)
    num_training_samples = num_samples - num_validation_samples
    training_dataset, validation_dataset = random_split(dataset, [num_training_samples, num_validation_samples])

    train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

    best_valid_loss = float('inf')
    patience_counter = 0
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        model.train()
        for batch_input, batch_target in train_loader:
            batch_input, batch_target = batch_input.to(device), batch_target.to(device)
            optimizer.zero_grad()

            predictions = model(batch_input)

            loss = loss_fn(predictions, batch_target)
            loss.backward()

            optimizer.step()

        print(f"  Training Loss: {loss.item()}")

        # Validation phase
        model.eval()
        with torch.no_grad():
            valid_loss = 0
            correct = 0
            total = 0
            for batch_input, batch_target in valid_loader:
                batch_input, batch_target = batch_input.to(device), batch_target.to(device)
                predictions = model(batch_input)
                _, predicted = torch.max(predictions, 1)
                total += batch_target.size(0)
                correct += (predicted == batch_target).sum().item()
                valid_loss += loss_fn(predictions, batch_target).item()

            accuracy = correct / total
            valid_loss /= len(valid_loader)
            print(f"  Validation Loss: {valid_loss}, Accuracy: {accuracy * 100:.2f}%")

            # Check for early stopping
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Early stopping after {patience} epochs of no improvement.")
                    return

        scheduler.step()

    print("Training finished.")

# function to calculate metrics
def calculate_metrics(model, dataset):
    data_loader = DataLoader(dataset, batch_size=16)
    model = model.to(device)
    model.eval()
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for batch_input, batch_target in data_loader:
            batch_input, batch_target = batch_input.to(device), batch_target.to(device)
            predictions = model(batch_input)
            all_predictions.append(predictions.argmax(dim=1))
            all_targets.append(batch_target)

    # all_predictions = torch.cat(all_predictions).cpu().numpy()
    # all_targets = torch.cat(all_targets).cpu().numpy()

    all_predictions = torch.cat(all_predictions)
    all_targets = torch.cat(all_targets)


    accuracy = accuracy_score(all_targets, all_predictions)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    # precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_predictions, average='weighted')
    # print(f"Precision: {precision:.4f}")
    # print(f"Recall: {recall:.4f}")
    # print(f"F1 Score: {f1:.4f}")

    # cm = confusion_matrix(all_targets, all_predictions)
    # # Plot the confusion matrix using Seaborn
    # sns.heatmap(cm, annot=True)
    # plt.title("Confusion Matrix")
    # plt.xlabel("Predicted Label")
    # plt.ylabel("True Label")
    # plt.show()

In [ ]:
# import torchattacks
import h5py
from torch.utils.data import TensorDataset
import time

# def generate_adversarial(dataset, model, model_name):
#     data_loader = DataLoader(dataset, batch_size=32)
#     model = model.to(device)

#     attack_dict = {}
#     attack_dict['fgsm'] = torchattacks.FGSM(model, eps=8/255)
#     attack_dict['fgsm'].set_mode_targeted_by_label()

#     attack_dict['bim'] = torchattacks.BIM(model, eps=8/255, alpha=2/255, steps=4)
#     attack_dict['bim'].set_mode_targeted_by_label()

#     attack_dict['pgd'] = torchattacks.PGD(model, eps=8/255, alpha=1/255, steps=4, random_start=True)
#     attack_dict['pgd'].set_mode_targeted_by_label()

#     attack_dict['cw'] = torchattacks.CW(model, c=1, kappa=0, steps=20, lr=0.01)
#     attack_dict['cw'].set_mode_targeted_by_label()

#     for name, attk in attack_dict.items():
#         st = time.time()
#         i = 0
#         current_batch = 0
#         h5_file_path = f'/content/drive/MyDrive/data/Target/DCASE19/{model_name}/{name}.h5'
#         with h5py.File(h5_file_path, 'a') as hf:
#             print(f"started {name}")
#             for batch_input, batch_target in data_loader:

#                 # generate adv image for image
#                 batch_input, batch_target = batch_input.to(device), batch_target.to(device)
#                 new_labels = torch.full((batch_target.size()), 0).to(device)
#                 # 0 = airport
#                 adv_images = attk(batch_input, new_labels)

#                 # write in h5 file
#                 for j in range(batch_input.shape[0]):
#                     grp_name = f'sample_{i}'
#                     if grp_name in hf:
#                         del hf[grp_name]  # Delete existing group
#                     grp = hf.create_group(f'sample_{i}')
#                     grp.create_dataset('data', data=adv_images[j].cpu().numpy())
#                     grp.create_dataset('label', data=batch_target[j].cpu())
#                     i += 1
#                 if (current_batch % 100 == 0):
#                     print(f"{current_batch} - ", end="")
#                 current_batch += 1
#             print("\n")

#         end = time.time()
#         print(f'{name} adv dataset for {model_name} stored, time {end-st}')

"""
runs the given model on all adv dataset of `adv_dir` directory
"""
def calculate_accuracy_for_adv_data(model, model_name, adv_dir):
    attk_list = ['cw', 'pgd', 'bim', 'fgsm']

    for attk in attk_list:
        h5_file_path = f'/content/drive/MyDrive/data/AdvData/DCASE19/{adv_dir}/{attk}.h5'
        data_list = []
        label_list = []

        with h5py.File(h5_file_path, 'r') as hf:
            # Iterate over the samples in the HDF5 file
            for sample_name in hf.keys():
                data = torch.tensor(hf[sample_name]['data'][:])
                label = hf[sample_name]['label'][()]

                # Append data and label to lists
                data_list.append(data)
                label_list.append(label)

        dataset_class = TensorDataset(torch.stack(data_list), torch.tensor(label_list))
        print(f'Stats for {model_name} on {adv_dir}/{attk} dataset', end=" ")
        calculate_metrics(model, dataset_class)
        del dataset_class
        del data_list
        del label_list


def for_all(model, model_name):
    adv_dirs = ['resnet50', 'resnet18', 'vit_base', 'mixer']
    for adv in adv_dirs:
        calculate_accuracy_for_adv_data(model, model_name, adv)

# ResNet 18

In [ ]:
import timm

resnet18 = timm.create_model('resnet18.a1_in1k', pretrained=True)
resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet18.fc = nn.Linear(in_features=512, out_features=10, bias=True)

In [ ]:
torch.cuda.empty_cache()
train_model(resnet18, dcase, learning_rate=1e-4, num_epochs=12, batch_size=128)

torch.save(resnet18.state_dict(), '/content/drive/MyDrive/data/Models/DCASE19/resnet18.pth')

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/resnet18.pth')
resnet18.load_state_dict(checkpoint)

generate_adversarial(dcase, resnet18, 'resnet18')

Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
started fgsm
0 - 100 - 200 - 300 - 400 - 

fgsm adv dataset for resnet18 stored, time 138.0474193096161
started bim
0 - 100 - 200 - 300 - 400 - 

bim adv dataset for resnet18 stored, time 218.81152892112732
started pgd
0 - 100 - 200 - 300 - 400 - 

pgd adv dataset for resnet18 stored, time 217.7463800907135
started cw
0 - 100 - 200 - 300 - 400 - 

cw adv dataset for resnet18 stored, time 275.1409556865692


In [ ]:
import timm

def ret_pred(model, dataset):
    data_loader = DataLoader(dataset, batch_size=16)
    model = model.to(device)
    model.eval()
    all_predictions = []
    with torch.no_grad():
        for batch_input, batch_target in data_loader:
            batch_input = batch_input.to(device)
            predictions = model(batch_input)
            all_predictions.append(predictions.argmax(dim=1))

    all_predictions = torch.cat(all_predictions)
    return all_predictions

def cal_ratio():
    attk_list = ['fgsm', 'bim', 'pgd', 'cw']
    adv_dir = 'resnet18'

    resnet18 = timm.create_model('resnet18.a1_in1k', pretrained=True)
    resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet18.fc = nn.Linear(in_features=512, out_features=10, bias=True)
    torch.cuda.empty_cache()
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/resnet18.pth')
    resnet18.load_state_dict(checkpoint)

    resnet50 = timm.create_model('resnet50.a1_in1k', pretrained=True)
    resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet50.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
    torch.cuda.empty_cache()
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/resnet50.pth')
    resnet50.load_state_dict(checkpoint)

    vit_base = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
    vit_base.patch_embed.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
    vit_base.head = nn.Linear(in_features=768, out_features=10, bias=True)
    torch.cuda.empty_cache()
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/vit_base.pth')
    vit_base.load_state_dict(checkpoint)

    mixer = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
    mixer.stem.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
    mixer.head = nn.Linear(in_features=768, out_features=10, bias=True)
    torch.cuda.empty_cache()
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/mixer.pth')
    mixer.load_state_dict(checkpoint)

    for attk in attk_list:
        h5_file_path = f'/content/drive/MyDrive/data/Target/DCASE19/{adv_dir}/{attk}.h5'
        data_list = []
        label_list = []

        with h5py.File(h5_file_path, 'r') as hf:
            # Iterate over the samples in the HDF5 file
            for sample_name in hf.keys():
                data = torch.tensor(hf[sample_name]['data'][:])
                label = hf[sample_name]['label'][()]

                # Append data and label to lists
                data_list.append(data)
                label_list.append(label)

        dataset_class = TensorDataset(torch.stack(data_list), torch.tensor(label_list))
        label_list = torch.tensor(label_list)


        r18 = ret_pred(resnet18, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(13859):
            if (r18[i] == label_list[i]):
                unfooled += 1
            else:
                same += 1

        print (unfooled/13859, same/13859, diff/13859)

        print(f'Ratio for {adv_dir}/{attk} dataset')

        r50 = ret_pred(resnet50, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(13859):
            if (r50[i] == label_list[i]):
                unfooled += 1
            elif (r50[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/13859, same/13859, diff/13859)

        vb = ret_pred(vit_base, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(13859):
            if (vb[i] == label_list[i]):
                unfooled += 1
            elif (vb[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/13859, same/13859, diff/13859)

        m = ret_pred(mixer, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(13859):
            if (m[i] == label_list[i]):
                unfooled += 1
            elif (m[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/13859, same/13859, diff/13859)

        del dataset_class
        del data_list
        del label_list

In [ ]:
cal_ratio()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


0.11414965004690093 0.8858503499530991 0.0
Ratio for resnet18/fgsm dataset
0.3371094595569666 0.20932246193809076 0.45356807850494263
0.8133342954037088 0.03651057074825023 0.150155133848041
0.8391658849844866 0.035356086297712676 0.1254780287178007
0.12172595425355365 0.8782740457464464 0.0
Ratio for resnet18/bim dataset
0.6111552060033192 0.1503715996825168 0.23847319431416408
0.883902157442817 0.02323399956706833 0.09286384299011473
0.9102388339707049 0.004690093080308825 0.08507107294898622
0.11559275561007287 0.8844072443899271 0.0
Ratio for resnet18/pgd dataset
0.5666353993794646 0.16372032614185728 0.2696442744786781
0.8799336171440941 0.023883397070495707 0.09618298578541021
0.9085071072948986 0.004112850855040046 0.08738004185006133
0.5681506602207952 0.43184933977920487 0.0
Ratio for resnet18/cw dataset
0.7937080597445703 0.074897178728624 0.13139476152680568
0.8999206291940255 0.026336676527888014 0.07374269427808644
0.9240926473771557 0.0056281116963705895 0.070279240926473

# Resnet 50

In [ ]:
import timm

resnet50 = timm.create_model('resnet50.a1_in1k', pretrained=True)
resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet50.fc = nn.Linear(in_features=2048, out_features=10, bias=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()
train_model(resnet50, dataset, learning_rate=1e-4, num_epochs=12, batch_size=64, stepSize=8)

torch.save(resnet50.state_dict(), '/content/drive/MyDrive/data/Models/DCASE19/resnet50.pth')

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/resnet50.pth')
resnet50.load_state_dict(checkpoint)

<All keys matched successfully>

In [ ]:
generate_adversarial(dcase, resnet50, 'resnet50')

Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
started fgsm
0 - 100 - 200 - 300 - 400 - 

fgsm adv dataset for resnet50 stored, time 202.45623302459717
started bim
0 - 100 - 200 - 300 - 400 - 

bim adv dataset for resnet50 stored, time 487.3192734718323
started pgd
0 - 100 - 200 - 300 - 400 - 

pgd adv dataset for resnet50 stored, time 463.3695569038391
started cw
0 - 100 - 200 - 300 - 400 - 

cw adv dataset for resnet50 stored, time 636.7308893203735


In [ ]:
import timm

def ret_pred(model, dataset):
    data_loader = DataLoader(dataset, batch_size=128, num_workers=2, pin_memory=True)
    model = model.to(device)
    model.eval()

    all_predictions_list = []
    with torch.no_grad():
        for batch_input, batch_target in data_loader:
            batch_input = batch_input.to(device)
            predictions = model(batch_input)
            all_predictions_list.append(predictions.argmax(dim=1))

    all_predictions = torch.cat(all_predictions_list)
    return all_predictions

def cal_ratio(size=13859, adv_dir='resnet50'):
    attk_list = ['fgsm', 'bim', 'pgd', 'cw']

    resnet18 = timm.create_model('resnet18.a1_in1k', pretrained=True)
    resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet18.fc = nn.Linear(in_features=512, out_features=10, bias=True)
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/resnet18.pth')
    resnet18.load_state_dict(checkpoint)

    resnet50 = timm.create_model('resnet50.a1_in1k', pretrained=True)
    resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet50.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/resnet50.pth')
    resnet50.load_state_dict(checkpoint)

    vit_base = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
    vit_base.patch_embed.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
    vit_base.head = nn.Linear(in_features=768, out_features=10, bias=True)
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/vit_base.pth')
    vit_base.load_state_dict(checkpoint)

    mixer = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
    mixer.stem.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
    mixer.head = nn.Linear(in_features=768, out_features=10, bias=True)
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/mixer.pth')
    mixer.load_state_dict(checkpoint)

    for attk in attk_list:
        h5_file_path = f'/content/drive/MyDrive/data/Target/DCASE19/{adv_dir}/{attk}.h5'
        data_list = []
        label_list = []

        with h5py.File(h5_file_path, 'r') as hf:
            # Iterate over the samples in the HDF5 file
            for sample_name in hf.keys():
                data = torch.tensor(hf[sample_name]['data'][:])
                label = hf[sample_name]['label'][()]

                # Append data and label to lists
                data_list.append(data)
                label_list.append(label)

        dataset_class = TensorDataset(torch.stack(data_list), torch.tensor(label_list))
        label_list = torch.tensor(label_list)

        r18 = ret_pred(resnet50, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(size):
            if (r18[i] == label_list[i]):
                unfooled += 1
            else:
                same += 1
        print(unfooled/size, same/size, diff/size)
        print(f'Ratio for {adv_dir}/{attk} dataset')

        r50 = ret_pred(resnet18, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(size):
            if (r50[i] == label_list[i]):
                unfooled += 1
            elif (r50[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print(unfooled/size, same/size, diff/size)

        vb = ret_pred(vit_base, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(size):
            if (vb[i] == label_list[i]):
                unfooled += 1
            elif (vb[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/size, same/size, diff/size)

        m = ret_pred(mixer, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(size):
            if (m[i] == label_list[i]):
                unfooled += 1
            elif (m[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/size, same/size, diff/size)

        del dataset_class
        del data_list
        del label_list

In [ ]:
cal_ratio(13859, 'resnet50')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


0.16429756836712606 0.8357024316328739 0.0
Ratio for resnet50/fgsm dataset
0.37831012338552567 0.13875459989898262 0.48293527671549175
0.8184573201529692 0.04199437188830363 0.1395483079587272
0.8608124684320658 0.031098924886355437 0.10808860668157876
0.15657695360415613 0.8434230463958439 0.0
Ratio for resnet50/bim dataset
0.6639728696154124 0.1016667869254636 0.23436034345912404
0.888015008297857 0.026048055415253627 0.08593693628688939
0.913630132044159 0.006061043365322173 0.0803088245905188
0.12663251316833826 0.8733674868316618 0.0
Ratio for resnet50/pgd dataset
0.6166390071433725 0.08434952016740024 0.29901147268922723
0.8819539649325349 0.024532794573923083 0.09351324049354211
0.9105274550833393 0.004185006133198644 0.08528753878346201
0.6408110253265026 0.35918897467349736 0.0
Ratio for resnet50/cw dataset
0.8047478173028357 0.07042355148279096 0.12482863121437333
0.9004978714192943 0.023161844288909734 0.07634028429179594


In [ ]:
# Ratio for resnet50/cw dataset
# 0.8047478173028357 0.07042355148279096 0.12482863121437333
# 0.9004978714192943 0.023161844288909734 0.07634028429179594
# 0.9251749765495346 0.0069990619813839385 0.06782596146908146

# VIT Base

In [ ]:
import timm

vit_base = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
vit_base.patch_embed.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
vit_base.head = nn.Linear(in_features=768, out_features=10, bias=True)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()
train_model(vit_base, dcase, learning_rate=1e-5, num_epochs=12, batch_size=32, patience=3)

torch.save(vit_base.state_dict(), '/content/drive/MyDrive/data/Models/DCASE19/vit_base.pth')

Epoch 1/12
  Training Loss: 1.529644250869751
  Validation Loss: 1.6833921122825009, Accuracy: 33.45%
Epoch 2/12
  Training Loss: 1.712351679801941
  Validation Loss: 1.5002490158738762, Accuracy: 41.25%
Epoch 3/12
  Training Loss: 1.1851561069488525
  Validation Loss: 1.3651209453056599, Accuracy: 47.96%
Epoch 4/12
  Training Loss: 1.138542652130127
  Validation Loss: 1.3282636835657318, Accuracy: 50.49%
Epoch 5/12
  Training Loss: 0.9792410135269165
  Validation Loss: 1.2010698544568028, Accuracy: 53.59%
Epoch 6/12
  Training Loss: 1.0341508388519287
  Validation Loss: 1.124776055072916, Accuracy: 58.14%
Epoch 7/12
  Training Loss: 0.899638295173645
  Validation Loss: 1.1451198575140416, Accuracy: 57.42%
Epoch 8/12
  Training Loss: 0.5624861121177673
  Validation Loss: 0.9694701556501717, Accuracy: 63.84%
Epoch 9/12
  Training Loss: 0.2540462613105774
  Validation Loss: 0.9296337739489544, Accuracy: 66.55%
Epoch 10/12
  Training Loss: 0.379729688167572
  Validation Loss: 0.9062014814

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/vit_base.pth')
vit_base.load_state_dict(checkpoint)

generate_adversarial(dcase, vit_base, 'vit_base')

Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
started fgsm
0 - 100 - 200 - 300 - 400 - 

fgsm adv dataset for vit_base stored, time 446.418824672699
started bim
0 - 100 - 200 - 300 - 400 - 

bim adv dataset for vit_base stored, time 1498.380660533905
started pgd
0 - 100 - 200 - 300 - 400 - 

pgd adv dataset for vit_base stored, time 1497.6290996074677
started cw
0 - 

In [ ]:
import timm

def ret_pred(model, dataset):
    data_loader = DataLoader(dataset, batch_size=128, num_workers=2, pin_memory=True)
    model = model.to(device)
    model.eval()
    all_predictions = []
    with torch.no_grad():
        for batch_input, batch_target in data_loader:
            batch_input = batch_input.to(device)
            predictions = model(batch_input)
            all_predictions.append(predictions.argmax(dim=1))

    all_predictions = torch.cat(all_predictions)
    return all_predictions

def cal_ratio():
    attk_list = ['fgsm', 'bim', 'pgd', 'cw']
    adv_dir = 'vit_base'
    torch.cuda.empty_cache()


    resnet18 = timm.create_model('resnet18.a1_in1k', pretrained=True)
    resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet18.fc = nn.Linear(in_features=512, out_features=10, bias=True)
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/resnet18.pth')
    resnet18.load_state_dict(checkpoint)

    resnet50 = timm.create_model('resnet50.a1_in1k', pretrained=True)
    resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet50.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/resnet50.pth')
    resnet50.load_state_dict(checkpoint)

    vit_base = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
    vit_base.patch_embed.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
    vit_base.head = nn.Linear(in_features=768, out_features=10, bias=True)
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/vit_base.pth')
    vit_base.load_state_dict(checkpoint)

    mixer = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
    mixer.stem.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
    mixer.head = nn.Linear(in_features=768, out_features=10, bias=True)
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/mixer.pth')
    mixer.load_state_dict(checkpoint)

    for attk in attk_list:
        h5_file_path = f'/content/drive/MyDrive/data/Target/DCASE19/{adv_dir}/{attk}.h5'
        data_list = []
        label_list = []

        with h5py.File(h5_file_path, 'r') as hf:
            # Iterate over the samples in the HDF5 file
            for sample_name in hf.keys():
                data = torch.tensor(hf[sample_name]['data'][:])
                label = hf[sample_name]['label'][()]

                # Append data and label to lists
                data_list.append(data)
                label_list.append(label)

        dataset_class = TensorDataset(torch.stack(data_list), torch.tensor(label_list))
        label_list = torch.tensor(label_list)

        r18 = ret_pred(vit_base, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(13859):
            if (r18[i] == label_list[i]):
                unfooled += 1
            else:
                same += 1
        print (unfooled/13859, same/13859, diff/13859)

        print(f'Ratio for {adv_dir}/{attk} dataset')

        r50 = ret_pred(resnet18, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(13859):
            if (r50[i] == label_list[i]):
                unfooled += 1
            elif (r50[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/13859, same/13859, diff/13859)

        vb = ret_pred(resnet50, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(13859):
            if (vb[i] == label_list[i]):
                unfooled += 1
            elif (vb[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/13859, same/13859, diff/13859)

        m = ret_pred(mixer, dataset_class)
        unfooled = 0
        diff = 0
        same = 0
        for i in range(13859):
            if (m[i] == label_list[i]):
                unfooled += 1
            elif (m[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/13859, same/13859, diff/13859)

        del dataset_class
        del data_list
        del label_list
    torch.cuda.empty_cache()


cal_ratio()

0.10563532722418645 0.8943646727758136 0.0
Ratio for vit_base/fgsm dataset
0.38372176924742046 0.1308896745796955 0.48538855617288407
0.36142578829641386 0.10808860668157876 0.5304856050220074
0.6621689876614474 0.06544483728984775 0.2723861750487048
0.1443105563171946 0.8556894436828054 0.0
Ratio for vit_base/bim dataset
0.6677970993578181 0.10101738942203622 0.23118551122014575
0.606898044591962 0.15592755610072878 0.23717439930730932
0.8783462010246049 0.02106934122231041 0.10058445775308464
0.17793491593910093 0.8220650840608991 0.0
Ratio for vit_base/pgd dataset
0.6097842557183057 0.09416263799696947 0.29605310628472475
0.5614402193520456 0.18543906486759507 0.25312071578035933
0.898044591961902 0.01176131033985136 0.09019409769824663
0.5291868100151526 0.4708131899848474 0.0
Ratio for vit_base/cw dataset
0.8017894508983332 0.05555956418211992 0.14265098491954686
0.7944296125261563 0.05794068836135363 0.14762969911249008
0.9207735045818601 0.006133198643480771 0.07309329677465906


# mixer

In [ ]:
import timm
mixer = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
mixer.stem.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
mixer.head = nn.Linear(in_features=768, out_features=10, bias=True)

model.safetensors:   0%|          | 0.00/240M [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()
train_model(mixer, dcase, learning_rate=1e-4, num_epochs=12, batch_size=64)

torch.save(mixer.state_dict(), '/content/drive/MyDrive/data/Models/DCASE19/mixer.pth')

Epoch 1/12
  Training Loss: 1.988483190536499
  Validation Loss: 1.7201913757757707, Accuracy: 37.89%
Epoch 2/12
  Training Loss: 1.606507420539856
  Validation Loss: 1.2947275394743138, Accuracy: 49.01%
Epoch 3/12
  Training Loss: 0.8739612102508545
  Validation Loss: 0.9987240609797564, Accuracy: 61.64%
Epoch 4/12
  Training Loss: 0.6760417819023132
  Validation Loss: 0.8281339555978775, Accuracy: 68.03%
Epoch 5/12
  Training Loss: 0.8595892786979675
  Validation Loss: 0.7224954177032817, Accuracy: 72.46%
Epoch 6/12
  Training Loss: 0.2867172062397003
  Validation Loss: 0.6901143409989097, Accuracy: 75.24%
Epoch 7/12
  Training Loss: 0.021618274971842766
  Validation Loss: 0.6682053411548788, Accuracy: 78.53%
Epoch 8/12
  Training Loss: 0.02592005580663681
  Validation Loss: 0.7515407946299423, Accuracy: 77.63%
Epoch 9/12
  Training Loss: 0.19544252753257751
  Validation Loss: 0.8770877454768528, Accuracy: 76.25%
Epoch 10/12
  Training Loss: 0.10138049721717834
  Validation Loss: 0.8

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/mixer.pth')
mixer.load_state_dict(checkpoint)

generate_adversarial(dcase, mixer, 'mixer')

Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
Attack mode is changed to 'targeted(label)'.
started fgsm
0 - 100 - 200 - 300 - 400 - 

fgsm adv dataset for mixer stored, time 321.79174995422363
started bim
0 - 100 - 200 - 300 - 400 - 

bim adv dataset for mixer stored, time 975.6368601322174
started pgd
0 - 100 - 200 - 300 - 

In [ ]:
import timm

def ret_pred(model, dataset):
    data_loader = DataLoader(dataset, batch_size=128, num_workers=2, pin_memory=True)
    model = model.to(device)
    model.eval()
    all_predictions = []
    with torch.no_grad():
        for batch_input, batch_target in data_loader:
            batch_input = batch_input.to(device)
            predictions = model(batch_input)
            all_predictions.append(predictions.argmax(dim=1))

    all_predictions = torch.cat(all_predictions)
    return all_predictions

def cal_ratio():
    attk_list = ['fgsm', 'bim', 'pgd', 'cw']
    adv_dir = 'mixer'
    torch.cuda.empty_cache()


    resnet18 = timm.create_model('resnet18.a1_in1k', pretrained=True)
    resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet18.fc = nn.Linear(in_features=512, out_features=10, bias=True)
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/resnet18.pth')
    resnet18.load_state_dict(checkpoint)

    resnet50 = timm.create_model('resnet50.a1_in1k', pretrained=True)
    resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    resnet50.fc = nn.Linear(in_features=2048, out_features=10, bias=True)
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/resnet50.pth')
    resnet50.load_state_dict(checkpoint)

    vit_base = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
    vit_base.patch_embed.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
    vit_base.head = nn.Linear(in_features=768, out_features=10, bias=True)
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/vit_base.pth')
    vit_base.load_state_dict(checkpoint)

    mixer = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
    mixer.stem.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
    mixer.head = nn.Linear(in_features=768, out_features=10, bias=True)
    checkpoint = torch.load('/content/drive/MyDrive/data/Models/DCASE19/mixer.pth')
    mixer.load_state_dict(checkpoint)


    for attk in attk_list:
        h5_file_path = f'/content/drive/MyDrive/data/Target/DCASE19/{adv_dir}/{attk}.h5'
        data_list = []
        label_list = []

        with h5py.File(h5_file_path, 'r') as hf:
            # Iterate over the samples in the HDF5 file
            for sample_name in hf.keys():
                data = torch.tensor(hf[sample_name]['data'][:])
                label = hf[sample_name]['label'][()]

                # Append data and label to lists
                data_list.append(data)
                label_list.append(label)

        dataset_class = TensorDataset(torch.stack(data_list), torch.tensor(label_list))
        label_list = torch.tensor(label_list)

        r18 = ret_pred(mixer, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(13859):
            if (r18[i] == label_list[i]):
                unfooled += 1
            else:
                same += 1
        print (unfooled/13859, same/13859, diff/13859)
        print(f'Ratio for {adv_dir}/{attk} dataset')

        r50 = ret_pred(resnet18, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(13859):
            if (r50[i] == label_list[i]):
                unfooled += 1
            elif (r50[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/13859, same/13859, diff/13859)

        vb = ret_pred(resnet50, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(13859):
            if (vb[i] == label_list[i]):
                unfooled += 1
            elif (vb[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/13859, same/13859, diff/13859)

        m = ret_pred(vit_base, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(13859):
            if (m[i] == label_list[i]):
                unfooled += 1
            elif (m[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/13859, same/13859, diff/13859)

        del dataset_class
        del data_list
        del label_list

cal_ratio()

0.06017750198427015 0.9398224980157298 0.0
Ratio for mixer/fgsm dataset
0.39750342737571254 0.15268056858359189 0.44981600404069555
0.38458763258532364 0.13485821487841837 0.48055415253625805
0.6188036654881305 0.11191283642398442 0.26928349808788515
0.10390360054838012 0.8960963994516199 0.0
Ratio for mixer/bim dataset
0.6807128941482069 0.09791471246121654 0.22137239339057652
0.6320080813911537 0.13471390432210117 0.23327801428674508
0.839093729706328 0.04892127859152897 0.11198499170214302
0.10498592972075907 0.895014070279241 0.0
Ratio for mixer/pgd dataset
0.6116602929504293 0.08694711018110975 0.3013925968684609
0.5605743560141424 0.16754455588426292 0.2718810881015946
0.8603073814849557 0.03932462659643553 0.10036799191860885
0.9207735045818601 0.07922649541813984 0.0
Ratio for mixer/cw dataset
0.8017894508983332 0.008947254491666065 0.18926329461000072
0.7944296125261563 0.010462515332996608 0.1951078721408471
0.5291868100151526 0.006133198643480771 0.4646799913413666
